<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Random Search + Grid Search로 Decision Tree 알고리즘 모델을 튜닝합니다.


# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [19]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format='retina'

In [20]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/income_happy.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [21]:
# 상위 몇 개 행 확인
data.head()

,income,happiness
0,3.862647,2.314489
1,4.979381,3.433490
2,4.923957,4.599373
3,3.214372,2.791114
4,7.196409,5.596398


In [22]:
# 기초통계량 확인
data.describe()

,income,happiness
count,498.000000,498.000000
mean,4.466902,3.392859
std,1.737527,1.432813
min,1.506275,0.266044
25%,3.006256,2.265864
50%,4.423710,3.472536
75%,5.991913,4.502621
max,7.481521,6.863388


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [23]:
# target 확인
target = 'happiness'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

**2) 학습용, 평가용 데이터 분리**

In [24]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

# 4.모델링


- RandomizedSearchCV를 사용해 최선의 하이퍼파라미터 값을 찾습니다.
- GridSearchCV를 사용해 앞에서 얻은 값 주변 값에 대한 성능을 확인합니다.

## 4.1.Randon Search

- RandomizedSearchCV를 사용해 최선의 하이퍼파라미터 값을 찾습니다.

In [25]:
# 1단계: 불러오기
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score

In [26]:
# 2단계: 선언하기
model_dt = DecisionTreeRegressor(random_state=2022)

# 파라미터 선언
  # max_depth: 1~20
param = {'max_depth': range(1, 21)}

# Random Search 선언
  # cv=5
  # n_iter=10
  # scoring='r2'
model = RandomizedSearchCV(model_dt,      # 기본 모델 이름
                           param,         # 앞에서 선언한 튜닝용 파라미터 변수
                           cv=5,          # k-Fold CV (default=5)
                           n_iter=10,     # Random하게 시도할 횟수 (default=10)
                           scoring='r2')  # 평가 방법

In [27]:
# 3단계: 학습하기
model.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=2022),
                   param_distributions={'max_depth': range(1, 21)},
                   scoring='r2')

In [28]:
# 선택된 파라미터와 성능 확인
print('최적파라미터:', model.best_params_)
print('성능:', model.best_score_)

최적파라미터: {'max_depth': 2}
성능: 0.6683626392532085


## 4.2.Grid Search

- GridSearchCV를 사용해 앞에서 얻은 값 주변 값에 대한 성능을 확인합니다.

In [29]:
# 2단계: 선언하기
# model_dt = DecisionTreeRegressor(random_state=2022)

# 파라미터 선언
param = {'max_depth': [1,2,3,4,5,6]}

# Grid Search 선언
  # cv=5
  # scoring='r2'
model = GridSearchCV(model_dt,
                     param,
                     cv=5,
                     scoring='r2')

In [31]:
# 3단계: 학습하기
model.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=2022),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6]}, scoring='r2')

In [32]:
# 선택된 파라미터와 성능 확인
print(model.best_params_)
print(model.best_score_)

{'max_depth': 3}
0.6777470417542905


## 4.3.예측과 평가

- 모델이 최적 파라미터 값으로 학습된 상태이니 바로 예측과 평가가 가능합니다.

In [33]:
# 4단계: 예측하기
y_pred = model.predict(x_test)

In [34]:
# 5단계: 평가하기
r2_score(y_test, y_pred)

0.7421459185473486